In [1]:
# Required Libraries Installation
!pip install tensorflow transformers gTTS moviepy SpeechRecognition pocketsphinx gTTS git+https://github.com/openai/whisper.git torchaudio


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-16oisizt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-16oisizt
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Importing Necessary Libraries
import moviepy.editor as mp
from google.colab import files
import whisper
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
import os

In [3]:
# Function to adjust audio speed
def adjust_audio_speed(video_file_path: str, audio_file_path: str, output_audio_path: str):
    video_clip = mp.VideoFileClip(video_file_path)
    video_duration = video_clip.duration
    audio_clip = mp.AudioFileClip(audio_file_path)

    # Print durations for debugging
    print(f"Video Duration: {video_duration} seconds")
    print(f"Audio Duration: {audio_clip.duration} seconds")

    adjusted_audio_clip = audio_clip.set_duration(video_duration)
    adjusted_audio_clip.write_audiofile(output_audio_path)
    video_clip.close()
    audio_clip.close()
    adjusted_audio_clip.close()

# Placeholder function for lip-sync model
def lip_sync_model(video_path, audio_path, output_video_path):
    video_clip = mp.VideoFileClip(video_path)
    audio_clip = mp.AudioFileClip(audio_path)
    video_with_audio = video_clip.set_audio(audio_clip)
    video_with_audio.write_videofile(output_video_path, codec='libx264')

# Function to translate text to a specified language
def translate_text(text, target_language):
    model_name = f"Helsinki-NLP/opus-mt-en-{target_language}"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    output = model.generate(input_ids)
    translated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_text

In [4]:
# Step 1: Upload the video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]  # Get the uploaded video file name
video = mp.VideoFileClip(video_path)

# Step 2: Extract and save audio from video
audio = video.audio
audio.write_audiofile("audio.wav")

# Step 3: Load and transcribe the audio file using a larger model
model = whisper.load_model("large")  # Use a larger model for better accuracy
audio_path = "audio.wav"
result = model.transcribe(audio_path)
transcribed_text = result["text"]
print("Transcribed Text: ", transcribed_text)

# Step 4: Emotion Detection
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion")
emotion_result = emotion_classifier(transcribed_text)
detected_emotion = emotion_result[0]['label']
print(f"Detected Emotion: {detected_emotion}")

# Step 5: Define target languages for translation
target_languages = ["fr", "es", "de", "hi"]  # French, Spanish, German, Hindi
translations = {}



Saving Deadpool & Wolverine _ Name For Myself _ Now Playing In Theaters.mp4 to Deadpool & Wolverine _ Name For Myself _ Now Playing In Theaters (5).mp4
MoviePy - Writing audio in audio.wav


MoviePy - Done.


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed Text:   You seem to have left out whether or not you had any experience as a member of a team. Get your team together for the movie event that's shattering records. And breaking the internet. I'm about to make a name for myself here. Who is your dialect coach? The Minions?


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

  warnings.warn(



Detected Emotion: joy


In [5]:
# Translate to each language
for lang in target_languages:
    translations[lang] = translate_text(transcribed_text, lang)
    print(f"Translation in {lang}: {translations[lang]}")

# Step 6: Combine translated audio with original video
for lang, translation in translations.items():
    from gtts import gTTS

    # Adjust TTS parameters based on detected emotion
    if detected_emotion == "joy":
        tts = gTTS(text=translation, lang=lang, slow=False)  # Normal speed
    elif detected_emotion == "sadness":
        tts = gTTS(text=translation, lang=lang, slow=True)  # Slower for sadness
    elif detected_emotion == "anger":
        tts = gTTS(text=translation, lang=lang, slow=True)  # Slower for anger
    else:
        tts = gTTS(text=translation, lang=lang, slow=False)  # Default speed

# Save the translated audio
tts.save(f"translated_audio_{lang}.mp3")

# Adjust the audio speed to match the video
adjust_audio_speed(video_path, f"translated_audio_{lang}.mp3", f"adjusted_audio_{lang}.mp3")

# Use the lip-sync model to create a video with synced audio
lip_sync_model(video_path, f"adjusted_audio_{lang}.mp3", f"dubbed_video_{lang}.mp4")

  warnings.warn("Recommended: pip install sacremoses.")



Translation in fr: Vous semblez avoir oublié si vous avez eu ou non une expérience en tant que membre d'une équipe. Rassemblez votre équipe pour l'événement de film qui est en train de briser des records. Et de briser l'Internet. Je suis sur le point de faire un nom pour moi-même ici. Qui est votre entraîneur de dialecte? Les Minions?
Translation in es: Pareces haber omitido si tenías o no alguna experiencia como miembro de un equipo, reunir a tu equipo para el evento de la película que está rompiendo récords y rompiendo Internet, estoy a punto de hacerme un nombre aquí, ¿quién es tu entrenador de dialectos, los Minions?
Translation in de: Sie scheinen ausgelassen zu haben, ob Sie irgendeine Erfahrung als Mitglied eines Teams hatten oder nicht. Holen Sie Ihr Team zusammen für das Filmereignis, das Rekorde zerbricht. Und brechen Sie das Internet. Ich bin dabei, einen Namen für mich hier zu machen. Wer ist Ihr Dialekt Trainer? Die Minions?
Translation in hi: आपको लगता है कि आप बाहर छोड़ 

MoviePy - Done.
Moviepy - Building video dubbed_video_hi.mp4.
MoviePy - Writing audio in dubbed_video_hiTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video dubbed_video_hi.mp4



t:  99%|█████████▉| 483/488 [00:12<00:00, 52.66it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file Deadpool & Wolverine _ Name For Myself _ Now Playing In Theaters (5).mp4, 1229760 bytes wanted but 0 bytes read,at frame 486/488, at time 20.27/20.32 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready dubbed_video_hi.mp4
